In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from threading import Thread
from func_timeout import func_timeout, FunctionTimedOut, func_set_timeout
from urllib.parse import urlparse
from collections import Counter
import urllib.request
from urllib.parse import urlparse
import whois

#Import the CSV
# csv = pd.read_csv("calls\call-c85aac6308a09d9112b7235ab11f329c3178fd29f457deda02aec99a924e4c81.csv")

In [ ]:
#Run this to get the complete DF merged from all functions!
# Ignore the 'ERRORtrying to connect to socket: closing socket' 
# The functions takes aprox 3 minutes to complete all the tasks

# createDF()

In [ ]:
@func_set_timeout(5)
def scanMediaInfo(siteUrl):
    #Empty list 
    allImageTags = []
    #empty dataframe
    dfImage = pd.DataFrame()

    content = requests.get(siteUrl).content
    # get soup
    soup = BeautifulSoup(content,'lxml') # choose lxml parser
    # find the tag : <img ... >
    image_tags = soup.findAll('img')
    #Counts all images
    totalImages = sum([ 1 for i in soup.find_all('img')])
    
    for image in image_tags:
        # CHECKS IF ALT IS EMPTY
        if(str(image.get('alt'))==""):
            #return NONE
            None
        else:
            # Else get the tag which is not empty
            tag = str(image.get('alt'))
        #Append the tag to the temporary list:(allImageTags)
        temp = allImageTags.append(tag)

    return totalImages, allImageTags

In [ ]:
@func_set_timeout(5)
def findContentLength(siteUrl):
    # set the url to perform the get request
    reqs = requests.get(siteUrl)
    # make a soup object by using beautiful soup and set the markup as html parser
    soup = BeautifulSoup(reqs.text, 'lxml')
    foundWords = []
    
    #-------- print and count all words
    #print("List of all the h1, h2, h3, h4, h5, h6 & p : \n")
    for heading in soup.body.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p"]):
        #print(heading.name + ' ' + heading.text.strip())
        foundWords.append(heading.text.strip().split())
        
    #print(foundWords)
    flatList = []
    
    #flatten the words 
    for sublist in foundWords:
        for singleWord in sublist:
            flatList.append(singleWord.lower())
    
    wordDensity = Counter(flatList)
    wordDensity = wordDensity.most_common()

    totalWords = str(len(flatList))
    
    return totalWords

In [ ]:
@func_set_timeout(5)
def getMetaDetails(siteUrl):
    #empty String (DO NOT REMOVE)
    metaTitle = ""
    try:
        external_sites_html = urllib.request.urlopen(siteUrl)
        websiteDomain = urlparse(siteUrl).netloc
        soup = BeautifulSoup(external_sites_html)
        metaTitle = soup.title.string.lstrip()

        # First get the meta description tag
        metaDescription = soup.find('meta', attrs={'name':'og:description'}) or soup.find('meta', attrs={'property':'description'}) or soup.find('meta', attrs={'name':'description'})

        # If description meta tag was found, then get the content attribute and save it to db entry
        if metaDescription:
            metaDescription = metaDescription.get('content')
    
    except:
        #SET NaN values if the code excepts
        metaTitle = "NaN"
        metaDescription = "NaN"
        
    return metaTitle, metaDescription
        
    

In [ ]:
def getSiteDetails2(csv):
    #Create an empty DF
    d = pd.DataFrame()

    for url in csv['Ur']:
        hostname = urlparse(url).netloc

        try:
            wordsTotal = findContentLength(url)
            numberImage = scanMediaInfo(url)[0]
            Alt = scanMediaInfo(url)[1]

            temp = pd.DataFrame({
                'Domain': hostname,
                'url': url,
                'Total Images': numberImage,
                'Image Tags': [Alt],
                'Content-length': wordsTotal,
            }, index=[0])

            d = pd.concat([d, temp])
        except:
            numberImage = 0
            Alt = "NaN"
            temp = pd.DataFrame({
                'Domain': hostname,
                'url': url,
                'Total Images': 0,
                'Image Tags': "NaN",
                'Content-length': wordsTotal,
            }, index=[0])

            d = pd.concat([d, temp])
            continue
    return d

In [ ]:
def getAllMeta(csv):
    #empty df
    emptyDf = pd.DataFrame()
    #start for-loop
    for url in csv['Ur']:
        try:
            #Get META-Title
            titleMeta = getMetaDetails(url)[0]
            #Get META-Description 
            metaDescriptionTag = getMetaDetails(url)[1]
            #Create TEMPORARY DF
            temp2 = pd.DataFrame({
                'url': url,
                'META-Title': titleMeta,
                'META-Description': metaDescriptionTag
            }, index=[0])

            emptyDf = pd.concat([emptyDf, temp2])
        except:
            numberImage = 0
            Alt = "NaN"
            #Create TEMPORARY DF
            temp2 = pd.DataFrame({
                'url': url,
                'META-Title': "NaN",
                'META-Description': "NaN"
            }, index=[0])

            emptyDf = pd.concat([emptyDf, temp2])
            continue
    return emptyDf

In [ ]:
def createDF(csv):
    #Get return from siteDetails2()
    dfList1 = getSiteDetails2(csv)
    #Get return from META Function for TITLE & Description
    dfList2 = getAllMeta(csv)
    # Get return from Domain Age finder
    dfList3 = getDomainAge(csv)
    #Merge the first 2 functions returns
    completeDf = pd.merge(dfList1,dfList2,left_on='url', right_on='url')
    #Merge the last return into the complete DF
    completeDf = pd.merge(completeDf,dfList3,left_on='url', right_on='url')
    
    #Complete DF Returned from all functions
    return completeDf

In [ ]:
def getDomainAge(csv):
    #Empty DF
    emptyDf2 = pd.DataFrame()
    
    for url in csv['Ur']:
        try:
            #Must have to get WHOIS information
            domain_name = url
            whois_info = whois.whois(domain_name)
            creationDate = whois_info.creation_date
            #temporary DF
            temp3 = pd.DataFrame({
                'url': url,               
                'Creation Date': creationDate
            }, index=[0])

            emptyDf2 = pd.concat([emptyDf2, temp3])
        except:
            #temporary DF
            temp3 = pd.DataFrame({
                'url': url,
                'Creation Date': "None"
            }, index=[0])

            emptyDf2 = pd.concat([emptyDf2, temp3])
            continue
    
    #Return the DF with Creation date of the domain.
    return emptyDf2